In [5]:
# experimenting with error channels
import numpy as np
from quantum_logical.error_channels import AmplitudeDamping, PhaseDampingS

ModuleNotFoundError: No module named 'quantum_logical.error_channels'

Operator-sum representation

\begin{align}
\epsilon(\rho) = \sum_i E_i \rho E_i^\dagger
\end{align}

Creating a noisy-gate

\begin{align}
U' = \sum_i E_i U
\end{align}

Amplitude damping
\begin{align}
E_0 &= \begin{pmatrix} 1 & 0 \\ 0 & \sqrt{1-\gamma} \end{pmatrix} \\
E_1 &= \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0 \end{pmatrix}
\end{align}


In [201]:
# create a noisy CNOT
from qiskit.circuit.library import HGate, CXGate, IGate
from qutip import Qobj

# create a noisy CNOT
gamma = 0.05
E0 = Qobj([[1, 0], [0, (1 - gamma) ** 0.5]])
E1 = Qobj([[0, (gamma / 2) ** 0.5], [0, 0]])

hadamard = HGate().to_matrix()
id = IGate().to_matrix()
cnot = CXGate().to_matrix()
gate = id

from qiskit.quantum_info import Operator

noisy_gate = Operator(E0.full() @ gate + E1.full() @ gate)
print(noisy_gate)

from qiskit.quantum_info import Kraus

kraus = Kraus(noisy_gate)

Operator([[1.        +0.j, 0.15811388+0.j],
          [0.        +0.j, 0.97467943+0.j]],
         input_dims=(2,), output_dims=(2,))


In [202]:
# fidelity metrics
from qiskit.quantum_info import Statevector, DensityMatrix

psi = Statevector([0, 1])
rho = DensityMatrix(psi)

noisy_rho = kraus(rho)
# normalize?

# fid = state_fidelity(rho, noisy_rho)
# print(f"Fidelity of states: {fid}")

#### State Fidelity

\begin{align}
F(\rho_1, \rho_2) = Tr[\sqrt{\sqrt{\rho_1}\rho_2\sqrt{\rho_1}}]^2
\end{align}

If one of the states is a pure state this simplifies to
$F(\rho_1, \rho_2) = \langle\psi_1|\rho_2|\psi_1\rangle$, where
$\rho_1 = |\psi_1\rangle\!\langle\psi_1|$.

#### Process Fidelity

\begin{align}
F*{\text{pro}}(\mathcal{E}, \mathcal{F})
= F(\rho*{\mathcal{E}}, \rho\_{\mathcal{F}})
\end{align}

#### Average gate fidelity

\begin{align}
F*{\text{ave}}(\mathcal{E}, U)
&= \int d\psi \langle\psi|U^\dagger
\mathcal{E}(|\psi\rangle\!\langle\psi|)U|\psi\rangle \\
&= \frac{d F*{\text{pro}}(\mathcal{E}, U) + 1}{d + 1}
\end{align}


In [203]:
# fidelity of states
from qiskit.quantum_info import (
    average_gate_fidelity,
    process_fidelity,
    gate_error,
    diamond_norm,
    state_fidelity,
    purity,
    concurrence,
    entropy,
    entanglement_of_formation,
    mutual_information,
)


input_state = np.array([0.0 + 0j, 1.0 + 0j])
input_state = input_state / np.linalg.norm(input_state)
psi = Statevector(input_state)


state0 = Statevector(gate @ psi.data)

# need to renormalize since noisy_gate is not unitary
state1 = noisy_gate @ psi.data
norm = np.linalg.norm(state1)
state1 = Statevector(state1 / norm)


fid = state_fidelity(state0, state1)
print(f"Fidelity of states: {fid}")
# TODO find minimum fidelity of states
# TODO find average fidelity of states
# depends on psi = Statevector([alpha, beta])


# gate fidelity
fid = average_gate_fidelity(channel=noisy_gate, target=HGate())
print(f"Gate Fidelity: {fid}")

Fidelity of states: 0.9743589743589741
Gate Fidelity: 0.3361373497422164


In [204]:
state0

Statevector([0.+0.j, 1.+0.j],
            dims=(2,))


In [205]:
state1

Statevector([0.16012815+0.j, 0.98709623+0.j],
            dims=(2,))


So the complexity I am thinking about is for these non-symmetric error channels, I can't talk about the fault rates for the gates without knowing the input state.

For example, if I have an amplitude damping channel with input state $|0\rangle$, then the fault rate is 0. During our circuits, we do not know input states to all gates, which makes it challenging to model the error correction process.
